# 🏭 CoT Factory Runner (Colab Edition)

Bu notebook, **Chain-of-Thought (CoT)** mimarisini kullanarak yüzlerce makaleyi toplu olarak işlemek için tasarlanmıştır.

**Özellikler:**
- ☁️ **Groq API** ile bulut tabanlı akıl yürütme.
- 🖼️ **Görsel Entegrasyon:** `visual_analysis.txt` (Grafik/Tablo verileri) varsa otomatik okur.
- ⚡ **Paralel Çalışma:** Bu notebook'tan birden fazla kopya açıp farklı aralıkları (Örn: 0-50, 51-100) aynı anda işleyebilirsiniz.

## 1️⃣ Kurulum ve Bağlantı

In [ ]:
# 1. Google Drive Bağla
from google.colab import drive
drive.mount('/content/drive')

# 2. Kütüphaneleri Yükle
!pip install groq pyyaml -q

import os
import sys
import glob
import time
import json
from google.colab import userdata

# 3. Proje Yolunu Tanımla (Kendi Drive yolunuzu kontrol edin!)
PROJECT_ROOT = "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST"
sys.path.append(PROJECT_ROOT)

# 4. Core Modülü İçe Aktar
try:
    from analysis.cot_laboratory.core.assembler import CoTAssembler
    print("✅ CoT Sistemi Başarıyla Yüklendi!")
except ImportError as e:
    print(f"❌ HATA: Proje yolu yanlış olabilir. Lütfen 'PROJECT_ROOT' değişkenini kontrol edin.\nDetay: {e}")

## 2️⃣ API Anahtarı

In [ ]:
# Sol menüdeki 'Secrets' (Anahtar ikonu) kısmına GROQ_API_KEY eklediğinizden emin olun.
try:
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    print("✅ API Anahtarı Alındı.")
except Exception:
    # Fallback: Elle giriş
    print("⚠️ Secrets'ta anahtar bulunamadı, manuel giriniz:")
    os.environ["GROQ_API_KEY"] = input("Groq API Key (gsk_...): ")

## 3️⃣ Fabrika Ayarları (Batch Config)

In [ ]:
# İşlenecek Tarifleri Seç
RECIPE_PATH = os.path.join(PROJECT_ROOT, "analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml")

# ------------------------------------------------------------------
# ⚡ PARALEL ÇALIŞMA AYARLARI
# Bir notebook'ta START=0, END=50 yapıp;
# Diğerinde START=51, END=100 yaparak hızı 2 katına çıkarabilirsiniz.
# ------------------------------------------------------------------
BATCH_START_INDEX = 0   # Başlangıç sırası
BATCH_END_INDEX = 200 # Bitiş sırası (Çok büyükse tümünü alır)
# ------------------------------------------------------------------

## 4️⃣ Üretimi Başlat (Run Factory)

In [ ]:
def get_pending_papers(root_dir):
    """İşlenecek makaleleri bulur."""
    data_dir = os.path.join(root_dir, "data", "processed_markdowns")
    # Recursive arama: data/processed_markdowns/**/O_ISAC_*.md
    search_pattern = os.path.join(data_dir, "**", "O_ISAC_*.md")
    all_candidates = glob.glob(search_pattern, recursive=True)
    return sorted(all_candidates)

def run_batch():
    print("🏗️ Fabrika Hazırlanıyor...")
    assembler = CoTAssembler(PROJECT_ROOT)
    
    all_papers = get_pending_papers(PROJECT_ROOT)
    
    # Slice list by batch settings
    target_papers = all_papers[BATCH_START_INDEX:BATCH_END_INDEX]
    
    print(f"📦 Toplam Aday Dosya: {len(all_papers)}")
    print(f"🎯 Hedef Aralığı: {BATCH_START_INDEX} - {BATCH_END_INDEX}")
    print(f"🚀 İşlenecek Dosya Sayısı: {len(target_papers)}")
    print("="*60)

    success_count = 0
    fail_count = 0

    for index, paper_path in enumerate(target_papers):
        # Global index for logging clarity
        global_idx = BATCH_START_INDEX + index
        
        filename = os.path.basename(paper_path)
        paper_id = os.path.splitext(filename)[0]
        
        print(f"\n[{global_idx}] 🦾 Processing {paper_id}...")
        
        try:
            # 1. Read Markdown
            with open(paper_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            if not content.strip():
                print(f"   ⚠️ SKIPPING: File empty.")
                continue

            # 2. Read Visual Analysis (DePlot/BLIP) if available
            paper_dir = os.path.dirname(paper_path)
            visual_path = os.path.join(paper_dir, "visual_analysis.txt")
            visual_content = None
            
            if os.path.exists(visual_path):
                with open(visual_path, 'r', encoding='utf-8') as vf:
                    visual_content = vf.read()
                print(f"   🖼️ Visual Data Loaded ({len(visual_content)} chars)")
            
            # 3. Execute CoT Extraction
            result = assembler.run_extraction(
                RECIPE_PATH, 
                content, 
                paper_id=paper_id, 
                visual_content=visual_content
            )
            
            if result.get("status") == "success":
                print(f"   ✅ SUCCESS")
                success_count += 1
            else:
                print(f"   ❌ FAILED: {result.get('error_message')}")
                fail_count += 1
                
            # API Rate Limit Protection
            time.sleep(1)
            
        except Exception as e:
            print(f"   💥 CRITICAL ERROR: {str(e)}")
            fail_count += 1

    print("="*60)
    print(f"🏁 Batch Completed.")
    print(f"✅ Success: {success_count}")
    print(f"❌ Failed:  {fail_count}")

# Run the Factory
run_batch()